In [1]:
from dataclasses import replace
from dnadb import dna, fasta, sample, taxonomy
from lmdbm import Lmdb
import matplotlib.pyplot as plt
import numpy as np
import numpy.typing as npt
import pandas as pd
from pathlib import Path
import re
import seaborn as sns
import sys
import tensorflow as tf
import tf_utilities as tfu
from tqdm.auto import tqdm
from typing import Iterable, Optional
import wandb

In [2]:
sys.path.append("../src")
from deepdna.nn.data_generators import SampleGenerator, _encode_sequences
from deepdna.nn.models import load_model, setbert

In [3]:
# tfu.devices.select_cpu()
tfu.devices.select_gpu(1)

2023-06-10 12:33:36.366310: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-10 12:33:36.366543: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-10 12:33:36.372051: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-10 12:33:36.372275: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-10 12:33:36.372454: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

([PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')],
 [PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')])

In [4]:
output_path = Path("/home/dwl2x/work/Datasets/Walker_Alex")
output_path.mkdir(exist_ok=True)

In [5]:
fasta_path = Path("/home/shared/walker_lab/alex/P_A_221205_cmfp.trim.contigs.pcr.good.unique.good.filter.unique.precluster.denovo.vsearch.pick.opti_mcc.0.03.pick.0.03.abund.0.03.pick.fasta")
otu_list_path = Path("/home/shared/walker_lab/digitalocean/Alex_SFD/shared_list/221205_cmfp.trim.contigs.pcr.good.unique.good.filter.unique.precluster.denovo.vsearch.asv.list")
otu_shared_path = Path("/home/shared/walker_lab/digitalocean/Alex_SFD/shared_list/221205_cmfp.trim.contigs.pcr.good.unique.good.filter.unique.precluster.denovo.vsearch.asv.shared")
metadata_path = Path("/home/shared/walker_lab/alex/230320_sfdspatial_meta_clean.csv")

## FASTA DB

In [6]:
def clean_entry(entry: fasta.FastaEntry):
    sequence = re.sub(r"[^" + dna.ALL_BASES + r"]", "", entry.sequence)
    return replace(entry, sequence=sequence)

In [7]:
# factory = fasta.FastaDbFactory(output_path / "alex.fasta.db")
# for entry in tqdm(map(clean_entry, fasta.entries(fasta_path))):
#     if len(entry) < 250:
#         continue
#     factory.write_entry(entry)
# factory.close()

In [8]:
fasta_db = fasta.FastaDb(output_path / "alex.fasta.db")

---

## Sample OTU Index

In [9]:
# with open(otu_list_path) as f:
#     keys = f.readline().strip().split('\t')
#     values = f.readline().strip().split('\t')
# otu_index = dict(zip(keys[2:], values[2:]))

In [10]:
# list(otu_index.values())[:5]

In [11]:
# factory = fasta.FastaIndexDbFactory(output_path / "alex.fasta.index.db")
# for i, asv in enumerate(tqdm(otu_index)):
#     fasta_id = otu_index[asv]
#     if fasta_id not in fasta_db:
#         continue
#     factory.write_entry(fasta_db[fasta_id], key=asv)
# factory.close()

In [12]:
index_db = fasta.FastaIndexDb(output_path / "alex.fasta.index.db")

---

## Load Metadata

In [13]:
metadata = pd.read_csv(metadata_path, index_col=0)
metadata

,taxon,ecomode,oo_present,date,county,site,clinical_signs,log_copy_number,gpsn,gpsw,...,swab_col,DNA_con,swab_plate_n,notes,richness,shannon,evenness,simpson,invsimpson,dis.sev
swab_label,,,,,,,,,,,,,,,,,,,,,
STP582C,Agkistrodon piscivorus,Aquatic,0,6/7/2015,Polk,Ouachita National Forest,NaN,0.00000,34.46294,-93.99638,...,1.0,3.06,1.0,Walker et al. ISME,618,4.804865,0.197578,0.974111,38.627160,Neg
STP633C,Crotalus horridus,Terrestrial,0,6/8/2015,Polk,Ouachita National Forest,NaN,0.00000,34.46262,-93.99853,...,1.0,2.18,1.0,Walker et al. ISME,486,4.505452,0.186233,0.962531,26.688501,Neg
STP634C,Agkistrodon piscivorus,Aquatic,0,6/8/2015,Polk,Ouachita National Forest,NaN,0.00000,34.57593,-94.03164,...,1.0,40.80,1.0,Walker et al. ISME,456,2.743417,0.034079,0.723965,3.622725,Neg
STP639C,Agkistrodon piscivorus,Aquatic,0,6/8/2015,Montgomery,Ouachita National Forest,NaN,0.00000,34.43632,-93.78750,...,1.0,3.90,1.0,Walker et al. ISME,659,4.919205,0.207730,0.975840,41.391414,Neg
STP640C,Agkistrodon piscivorus,Aquatic,0,6/8/2015,Montgomery,Ouachita National Forest,NaN,0.00000,34.43632,-93.78750,...,1.0,6.14,1.0,Walker et al. ISME,384,4.312839,0.194407,0.963434,27.347450,Neg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
STP1502C,Nerodia sipedon,Aquatic,1,NaN,Overton,Waterloo Falls,NaN,0.30103,NaN,NaN,...,11.0,22.60,2.0,Walker et al. ISME,692,4.422581,0.120392,0.959926,24.954109,Low
STP1503C,Nerodia sipedon,Aquatic,1,NaN,Overton,Waterloo Falls,NaN,0.30103,NaN,NaN,...,11.0,22.60,2.0,Walker et al. ISME,823,5.162691,0.212192,0.980217,50.549217,Low
STP762C,Thamnophis sirtalis,Terrestrial,1,NaN,Cumberland,Catoosa WMA,NaN,1.00000,NaN,NaN,...,2.0,23.20,1.0,Walker et al. ISME - Austin Property-Crossville,739,4.925323,0.186379,0.979888,49.721213,Low


In [14]:
# metadata_sample_names = set(metadata.index.to_list())

---

## Sample Mapping

In [15]:
# with open(otu_shared_path) as f:
#     header = f.readline().strip().split('\t')
#     lines = [line.strip().split('\t') for line in tqdm(f)]
# len(lines)

In [16]:
# cols = [i for i in range(3, len(header)) if index_db.contains_key(header[i])]

In [17]:
# header[:5]

In [18]:
# index_db.key_to_fasta_id("ASV0000004")

In [19]:
# indices = [i for i in tqdm(range(3, len(header))) if index_db.contains_key(header[i])]

In [20]:
# fasta_ids = {i: index_db.key_to_fasta_id(header[i]) for i in tqdm(indices)}

In [21]:
# factory = sample.SampleMappingDbFactory(output_path / "alex.fasta.mapping.db")
# for row in tqdm(lines):
#     sample_name = row[1]
#     if sample_name not in metadata_sample_names:
#         continue
#     sample_factory = sample.SampleMappingEntryFactory(sample_name, index_db)
#     for i in indices:
#         if (count := int(row[i])) == 0:
#             continue
#         fasta_id = fasta_ids[i]
#         sample_factory.add_entry(fasta_db[fasta_id], count)
#     factory.write_entry(sample_factory.build())
# factory.close()

In [22]:
mapping = sample.SampleMappingDb(output_path / "alex.fasta.mapping.db", index_db)

In [23]:
sample_names = set([sample.name for sample in mapping])
len(sample_names)

737

## Taxonomy

In [24]:
tax_data = pd.read_csv("/home/shared/walker_lab/alex/230428_SFDtaxfinal_engineer.csv")
tax_data

,otu,kingdom,phylum,class,order,family,genus
0,Otu000002,Bacteria,Actinobacteria,Actinobacteria,Micrococcales,Intrasporangiaceae,Janibacter
1,Otu000003,Bacteria,Deinococcus-Thermus,Deinococci,Deinococcales,Deinococcaceae,Deinococcus
2,Otu000004,Bacteria,Proteobacteria,Gammaproteobacteria,Betaproteobacteriales,Burkholderiaceae,Burkholderiaceae_unclassified
3,Otu000005,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter
4,Otu000006,Bacteria,Proteobacteria,Gammaproteobacteria,Betaproteobacteriales,Burkholderiaceae,Burkholderiaceae_unclassified
...,...,...,...,...,...,...,...
25047,Otu069105,Bacteria,Proteobacteria,Gammaproteobacteria,Gammaproteobacteria_unclassified,Gammaproteobacteria_unclassified,Gammaproteobacteria_unclassified
25048,Otu069123,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacteriales,Enterobacteriaceae,Enterobacteriaceae_unclassified
25049,Otu069172,Bacteria,Actinobacteria,Actinobacteria,Micrococcales,Microbacteriaceae,Microbacteriaceae_unclassified
25050,Otu069191,Bacteria,Proteobacteria,Gammaproteobacteria,Betaproteobacteriales,Betaproteobacteriales_unclassified,Betaproteobacteriales_unclassified


In [25]:
tax_dict = {}
for _, row in tax_data.iterrows():
    otu, taxons = row[0], taxonomy.join_taxonomy(tuple(row.values[1:]))
    tax_dict[otu] = taxons

In [26]:
tax_dict["Otu000002"]

'k__Bacteria; p__Actinobacteria; c__Actinobacteria; o__Micrococcales; f__Intrasporangiaceae; g__Janibacter; s__'

In [27]:
asv_to_otu = {}
for entry in fasta_db:
    otu_id = entry.extra.split()[1]
    assert otu_id.startswith("Otu")
    asv_to_otu[entry.identifier] = otu_id
len(asv_to_otu)

1883435

---

## Pathogen Load TSVs

In [53]:
disease_severity = {}
disease_ids = {"Neg": 0, "Low": 1, "Moderate": 2, "Severe": 3}
for index, row in metadata.iterrows():
    if index not in sample_names:
        continue
    disease_severity[index] = disease_ids[row["dis.sev"]]

In [60]:
pathogen_loads = pd.read_csv(output_path / "alex.pathogen_load.tsv", sep="\t", index_col=0)["pathogen_load"].astype(np.float32).to_dict()

In [280]:
# with open(output_path / "alex.pathogen_load.tsv", "w") as f:
#     f.write(f"label\tpathogen_load\n")
#     for index, row in metadata.iterrows():
#         if index not in sample_names:
#             continue
#         f.write(f"{index}\t{row['log_copy_number']}\n")

---

## Data Generator

In [281]:
# p = pd.read_csv(output_path / "alex.pathogen_load.tsv", sep="\t", index_col=0)
# p

In [282]:
from typing import Generic, TypeVar

_T = TypeVar("_T")

class SampleTargetGenerator(SampleGenerator, Generic[_T]):
    def __init__(
        self,
        samples: Iterable[sample.SampleInterface],
        targets: Iterable[dict[str, _T]],
        sequence_length: int,
        kmer: int = 1,
        subsample_size: int|None = None,
        augment_slide: bool = True,
        augment_ambiguous_bases: bool = True,
        batch_size: int = 32,
        batches_per_epoch: int = 100,
        class_weights: Optional[npt.ArrayLike] = None,
        shuffle: bool = True,
        balance: bool = False,
        rng: np.random.Generator = np.random.default_rng()
    ):
        super().__init__(
            samples=samples,
            sequence_length=sequence_length,
            kmer=kmer,
            subsample_size=subsample_size,
            augment_slide=augment_slide,
            augment_ambiguous_bases=augment_ambiguous_bases,
            batch_size=batch_size,
            batches_per_epoch=batches_per_epoch,
            class_weights=class_weights,
            shuffle=shuffle,
            balance=balance,
            rng=rng
        )
        self.targets = tuple(targets)

    def generate_batch(self, rng: np.random.Generator):
        subsample_size = self.subsample_size or 1
        sequence_ids = [None] * self.batch_size
        sequences = np.empty((self.batch_size, subsample_size), dtype=f"<U{self.sequence_length}")
        targets = tuple([] for _ in self.targets)
        sample_names = []
        samples = self.sample_sampler.sample(self.batch_size, self.balance, rng)
        for i, sample in enumerate(samples):
            sample_names.append(sample.name)
            sequence_ids[i], sequences[i] = zip(*self.sequence_sampler.sample_with_ids(sample, subsample_size, rng))
            for j, target in enumerate(self.targets):
                targets[j].append(target[sample.name])
        sequences = _encode_sequences(sequences, self.augment_ambiguous_bases, self.rng)
        if self.subsample_size is None:
            sequences = np.squeeze(sequences, axis=1) # type: ignore
        if self.kmer > 1:
            sequences = dna.encode_kmers(sequences, self.kmer, not self.augment_ambiguous_bases)
        x, y = sequences.astype(np.int32), tuple(map(np.array, targets))
        if len(y) == 1:
            y = y[0]
        return sample_names, sequence_ids, x, y

    def reduce_batch(self, batch):
        return batch[2:]

## Construct Training/Validation Sets

In [283]:
rng = np.random.default_rng(0)

In [284]:
samples = [sample.DemultiplexedFastaSample(fasta_db, entry) for entry in mapping]

In [285]:
rng.shuffle(samples)
split_index = int(0.2*len(samples))
train_samples = samples[split_index:]
val_samples = samples[:split_index]

train_pathogen_loads = {sample.name: pathogen_loads[sample.name] for sample in train_samples}
val_pathogen_loads = {sample.name: pathogen_loads[sample.name] for sample in val_samples}

train_disease_severity = {sample.name: disease_severity[sample.name] for sample in train_samples}
val_disease_severity = {sample.name: disease_severity[sample.name] for sample in val_samples}
len(train_pathogen_loads), len(val_pathogen_loads), len(train_disease_severity), len(val_disease_severity)

(590, 147, 590, 147)

In [288]:
def compute_class_weights(samples, targets):
    classes, counts = np.unique(list(targets.values()), return_counts=True)
    return 1.0/len(classes)/counts[np.array([targets[s.name] for s in samples])]

In [289]:
# Test (should be ~= 250 each)
np.unique([
    train_disease_severity[s.name] \
    for s in rng.choice(train_samples, 1000, p=compute_class_weights(train_samples, train_disease_severity))
], return_counts=True)

/tmp/ipykernel_457341/3605520578.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  for s in rng.choice(train_samples, 1000, p=compute_class_weights(train_samples, train_disease_severity))


(array([0, 1, 2, 3]), array([231, 264, 273, 232]))

In [290]:
train_targets = [train_pathogen_loads, train_disease_severity]
val_targets = [val_pathogen_loads, val_disease_severity]

In [326]:
common_args = dict(
    sequence_length = 250,
    kmer = 3,
    subsample_size = 1000,
    batch_size=16
)

train_dataset = SampleTargetGenerator(
    train_samples,
    train_targets,
    class_weights=compute_class_weights(train_samples, train_disease_severity),
    batches_per_epoch=20,
    **common_args
)

val_dataset = SampleTargetGenerator(
    val_samples,
    val_targets,
    class_weights=compute_class_weights(val_samples, val_disease_severity),
    batches_per_epoch=16,
    **common_args
)

In [327]:
train_dataset[0][1]

(array([3.91797781, 0.        , 3.73575854, 0.90309   , 1.46239805,
        3.11693954, 2.52504492, 1.59106457, 0.69897002, 0.        ,
        3.55059528, 1.        , 0.30103001, 0.        , 1.25527251,
        1.81954396]),
 array([3, 0, 3, 1, 2, 3, 3, 2, 1, 0, 3, 1, 1, 0, 2, 2]))

## Model Training

In [322]:
# api = wandb.Api()
run = wandb.init(project="setbert-pathogen-load", name="severity-classification")

In [323]:
# path = api.artifact("sirdavidludwig/setbert-pretrain/setbert-pretrain-walker-abund-128d-250l:v0").download()
path = run.use_artifact("sirdavidludwig/setbert-pretrain/setbert-pretrain-walker-abund-128d-250l:v2").download()
setbert_encoder = setbert.SetBertEncoderModel(
    load_model(path, setbert.SetBertPretrainModel).base
)
setbert_encoder.base.dnabert_encoder.chunk_size = 256

wandb: Downloading large artifact setbert-pretrain-walker-abund-128d-250l:v2, 91.05MB. 4 files... 
wandb:   4 of 4 files downloaded.  
Done. 0:0:0.1


In [324]:
# setbert_encoder(train_dataset[0][0])

In [325]:
num_classes = len(set(disease_severity.values()))
num_classes

4

In [328]:
y = x = tf.keras.layers.Input((1000, 248))
y = setbert_encoder(y)
y_pathogen_load = tf.keras.layers.Dense(1, activation="softplus", name="pathogen_load")(y)
y_disease_severity = tf.keras.layers.Dense(num_classes, activation="softmax", name="disease_severity")(y)
model = tf.keras.Model(x, (y_pathogen_load, y_disease_severity))
model.compile(
    loss={
        "pathogen_load": tf.keras.losses.MeanSquaredError(name="pathogen_load_loss"),
        "disease_severity": tf.keras.losses.SparseCategoricalCrossentropy(name="disease_severity_loss", from_logits=False)
    },
    metrics={
        "disease_severity": tf.keras.metrics.SparseCategoricalAccuracy(name="disease_severity_accuracy")
    },
    optimizer=tf.keras.optimizers.Adam(1e-4)
)
model.summary()

Model: "model_36"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_38 (InputLayer)          [(None, 1000, 248)]  0           []                               
                                                                                                  
 set_bert_encoder_model_3 (SetB  ((None, 128),       9240192     ['input_38[0][0]']               
 ertEncoderModel)                [(None, 8, 1001, 1                                               
                                001),                                                             
                                 (None, 8, 1001, 10                                               
                                01),                                                              
                                 (None, 8, 1001, 10                                        

In [329]:
wandb_callback = wandb.keras.WandbCallback(save_model=False)
wandb_callback.save_model_as_artifact = False

In [330]:
checkpoint = tf.keras.callbacks.ModelCheckpoint("./alex_severity", save_best_only=True)

In [331]:
model.fit(train_dataset, validation_data=val_dataset, epochs=100, callbacks=[wandb_callback, checkpoint])

Epoch 1/100
20/20 [==============================] - ETA: 0s - loss: 13.7367 - pathogen_load_loss: 9.1432 - disease_severity_loss: 4.5934 - disease_severity_disease_severity_accuracy: 0.2625 WARNING:tensorflow:5 out of the last 5 calls to <function Model.make_test_function.<locals>.test_function at 0x7f0807dff400> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


INFO:tensorflow:Assets written to: ./alex_severity/assets


INFO:tensorflow:Assets written to: ./alex_severity/assets
/opt/conda/lib/python3.10/site-packages/keras/engine/functional.py:1384: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


20/20 [==============================] - 561s 28s/step - loss: 13.7367 - pathogen_load_loss: 9.1432 - disease_severity_loss: 4.5934 - disease_severity_disease_severity_accuracy: 0.2625 - val_loss: 8.0196 - val_pathogen_load_loss: 3.1977 - val_disease_severity_loss: 4.8218 - val_disease_severity_disease_severity_accuracy: 0.2148
Epoch 2/100


/opt/conda/lib/python3.10/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


...

INFO:tensorflow:Assets written to: ./alex_severity/assets


INFO:tensorflow:Assets written to: ./alex_severity/assets


20/20 [==============================] - 553s 28s/step - loss: 4.9760 - pathogen_load_loss: 3.7914 - disease_severity_loss: 1.1846 - disease_severity_disease_severity_accuracy: 0.4688 - val_loss: 5.7959 - val_pathogen_load_loss: 3.1913 - val_disease_severity_loss: 2.6045 - val_disease_severity_disease_severity_accuracy: 0.1758


/opt/conda/lib/python3.10/site-packages/keras/engine/functional.py:1384: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/opt/conda/lib/python3.10/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


Epoch 11/100
20/20 [==============================] - 515s 26s/step - loss: 4.7350 - pathogen_load_loss: 3.7023 - disease_severity_loss: 1.0327 - disease_severity_disease_severity_accuracy: 0.5500 - val_loss: 6.0809 - val_pathogen_load_loss: 3.5684 - val_disease_severity_loss: 2.5125 - val_disease_severity_disease_severity_accuracy: 0.2227
Epoch 12/100
20/20 [==============================] - 515s 26s/step - loss: 4.8323 - pathogen_load_loss: 3.8335 - disease_severity_loss: 0.9988 - disease_severity_disease_severity_accuracy: 0.5500 - val_loss: 6.0564 - val_pathogen_load_loss: 3.1679 - val_disease_severity_loss: 2.8885 - val_disease_severity_disease_severity_accuracy: 0.2188
Epoch 13/100
20/20 [==============================] - 515s 26s/step - loss: 4.5677 - pathogen_load_loss: 3.6006 - disease_severity_loss: 0.9671 - disease_severity_disease_severity_accuracy: 0.5625 - val_loss: 6.4038 - val_pathogen_load_loss: 3.8558 - val_disease_severity_loss: 2.5480 - val_disease_severity_disease_